In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import os
os.chdir('C:\\Users\\nbg05\\Local Documents\\Projects\WhiteHeaven')

import puzzle.lib.puzzletools as tools
from puzzle.lib.puzzlesolving import PuzzlePiece, Puzzle

from ipywidgets import interact
from tqdm import tqdm

In [2]:
# create a list of all the images in the folder
# Gets the puzzle piece image
def get_piece_image(path):
    # images are of the format piece#.png
    img = cv2.imread(path)
    return img

piece_dir = 'dataset/pieces-clean'
image_files = os.listdir(piece_dir)
image_files = sorted(image_files, key=lambda x: int(x[5:-4]))

# load all off the images
piece_images = []
for i in tqdm(range(len(image_files)), 'images'):
    piece_images.append( get_piece_image(os.path.join(piece_dir, image_files[i])) )

piece_contours = []
for i in tqdm(range(len(piece_images)), 'contours'):
    piece_contours.append(tools.get_contour(piece_images[i], 226))

contours: 100%|██████████| 1000/1000 [00:00<00:00, 6820.79it/s]


In [3]:
pieces = []
for i in tqdm(range(len(piece_contours))):
    pieces.append(PuzzlePiece(piece_contours[i]))

100%|██████████| 1000/1000 [00:46<00:00, 21.65it/s]


In [6]:
num_inner = 0
num_outer = 0
num_flat = 0
for piece in pieces:
    for edge in piece.edges:
        if edge.type == 'inner':
            num_inner += 1
        elif edge.type == 'outer':
            num_outer += 1
        elif edge.type == 'flat':
            num_flat += 1

print(num_inner, num_outer, num_flat)

1935 1935 130


In [ ]:
puzzle = Puzzle(pieces, (40, 25))
puzzle.place_piece(0, 0, 0, 2)
puzzle.place_piece(1, 0, 42, 3)
puzzle.place_piece(0, 1, 90, 0)

In [12]:
@interact(x=(0, 24), y=(0, 40), piece=(0, len(pieces)-1), j=(0, 3))
def visualize_match(x, y, piece, j):
    edges = puzzle.get_pocket(x, y)
    piece_edges = pieces[piece].edges
    piece_edges = piece_edges[j:] + piece_edges[:j]

    fig, ax = plt.subplots(2, 2)
    fig.set_size_inches(13, 8)
    ax = ax.flatten()

    for i in range(4):
        ax[i].plot(piece_edges[i].normalized[:, 0], piece_edges[i].normalized[:, 1])
        if edges[i].points is None:
            continue
        ax[i].plot(edges[i].normalized[:, 0], edges[i].normalized[:, 1])
    
    plt.show()


interactive(children=(IntSlider(value=12, description='x', max=24), IntSlider(value=20, description='y', max=4…

In [ ]:
x, y = 24, 0
indices, scores = puzzle.solve_piece(x, y)

@interact(i=(0,9))
def visualize(i=0):
    print(indices[i])
    print(scores[i])
    visualize_match(x, y, indices[i, 0], indices[i, 1])

In [ ]:
def visualize_score(x, y, piece, j, i):
    edges = puzzle.get_pocket(x, y)
    piece_edges = pieces[piece].edges
    piece_edges = piece_edges[j:] + piece_edges[:j]

    fig, ax = plt.subplots()
    ax.set_ylim(0, 4)

    if edges[i].points is None:
        return
    
    distances = piece_edges[i].kd_tree.query(edges[i].normalized, k=1)[0]
    ax.plot(distances)
    print(np.sum(np.maximum(0, distances-1))/len(distances))
    
    plt.show()

@interact(i=(0,9), j=(0,3))
def visualize_scores(i=0, j=3):
    visualize_score(x, y, indices[i, 0], indices[i, 1], j)

In [ ]:
i = 0
puzzle.place_piece(x, y, indices[i, 0], indices[i, 1])

In [4]:
points = np.array([(x, y) for x in range(25) for y in range(2)])
points = np.concatenate((points, np.array([(x, y) for y in range(2, 40) for x in range(25)])))
print(len(points))
print(points)

1000
[[ 0  0]
 [ 0  1]
 [ 1  0]
 ...
 [22 39]
 [23 39]
 [24 39]]


In [21]:
# solve the first 50 points
puzzle = Puzzle(pieces, (40, 25))
puzzle.place_piece(0, 0, 0, 2)

guesses = np.zeros(points.shape[0], dtype=np.int32)
i = 1
best = 0
while i != 0 and i < 50:
    if i > best:
        best = i
        print(i)
    
    x, y = points[i]
    puzzle.remove_piece(x, y)
    indices, scores = puzzle.solve_piece(x, y)
    
    confidence = scores[1]-scores[0]
    #print("{}: confidence: {:.3f}, score: {:.3f}".format(i, confidence, scores[0]))
    #print(i, scores[:5])
    
    if scores[guesses[i]] > 0.8 and confidence < 0.1:
        # mistake was made
        guesses[i] = 0
        i -= 1
        guesses[i] += 1
        continue
    puzzle.place_piece(x, y, indices[guesses[i], 0], indices[guesses[i], 1])
    i += 1
    

1
2
3
4
5
6
7
8


KeyboardInterrupt: 

In [ ]:
while i > 47:
    x, y = points[i]
    puzzle.remove_piece(x, y)
    guesses[i] = 0
    i -= 1
i += 1

In [14]:
@interact(i=(0, 100))
def visualize_puzzle(i):
    x, y = points[i]
    visualize_match(x, y, puzzle.board[y, x], puzzle.rotations[y, x])

interactive(children=(IntSlider(value=50, description='i'), Output()), _dom_classes=('widget-interact',))

In [22]:
import pandas as pd
df = pd.DataFrame(puzzle.board.T)
df

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0,90,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,42,290,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,120,813,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,77,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
